# Matches_Predicao_RL

Este notebook é dedicado à predição de dados da tabela **Matches** fornecida pela IBM, que contém dados detalhados sobre as partidas, incluindo informações como quantidade de gols nas partidas, cartões amarelos que têm no total, entre outros. O objetivo deste notebook é confirmar as hipóteses e prever mais informações com os dados dessa tabela.

## Objetivo

O objetivo deste notebook é fornecer uma predição de dados das partidas da série A do Campeonato Brasileiro, ajudando a identificar padrões e tendências que possam ser úteis para diversas aplicações, como previsões de resultados e desempenho dos times.

## Como Usar Este Notebook

1. **Configuração do Ambiente**:
   - **Google Colab**: No Google Colab, será necessário fazer o upload das tabelas para o Google Drive e montar o drive no notebook.
   - **Localmente**: Se for rodar o notebook localmente, é necessário baixar as tabelas e colocá-las no mesmo diretório do notebook ou ajustar os caminhos dos arquivos conforme necessário.

2. **Instalação de Dependências**:
   - Certifique-se de que todas as bibliotecas necessárias estão instaladas. Você pode instalar as dependências utilizando o seguinte comando:
     ```python
     !pip install -r requirements.txt
     ```

3. **Execução do Notebook**:
   - Antes de rodar esse notebook, execute por completo o notebook `matches_tratado.ipynb` para que sejam exportadas em suas células a
   respectiva tabela contendo os dados tratados, que serão utilizados neste notebook.
   - Siga as células de código sequencialmente para garantir que todas as etapas sejam executadas corretamente. Cada seção do notebook está organizada para facilitar a compreensão e a análise dos dados.

## Nesse Notebook Será Abordado

1. **Modelagem para o problema**:
   - Apresentar o modelo candidato - qual modelo estamos utilizando e para que;
   - Métricas relacionadas ao modelo - avaliação de desempenho.
   - Hiperparâmetros - aplicação de GridSearch

# Dependências
Para rodar o notebook de forma local, é recomendado que inicie uma venv (ambiente virtual) e instale as dependências.

Se estiver utilizando o Google Colab, pule esta etapa.


# Instala as dependências

In [ ]:
# Instala as dependências
!pip install -r requirements.txt

# Importando bibliotecas

Aqui é importado as dependências necessárias para a executação do projeto.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, confusion_matrix, roc_curve, roc_auc_score, RocCurveDisplay, auc
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
 

# Carregando o Dataset
Carrega o CSV da tabela matches tratado para criar um dataframe.

In [3]:
df = pd.read_csv('../../notebooks/data/tratado/matches_tratado.csv')

# 1. Segundo Modelo Candidato

## 1.1 Regressão Logística

### 1.2 Hipótese do Modelo: Qual a probabilidade de um time ganhar com certas variáveis.

##### Premissa Principal:
O objetivo de previsão desse modelo é que o desempenho passado de um time, medido por diversas métricas estatísticas (como posse de bola, número de chutes a gol, cartões, etc.), pode prever a probabilidade de um time vencer em uma partida futura. O modelo utiliza uma combinação de variáveis pré-jogo e dados históricos dos times para calcular essas probabilidades.

##### Justificativa da Premissa:
1. **Posse de bola e estatísticas de ataque e defesa** são fortes indicativos de domínio em uma partida. Times que têm alta posse de bola e boa eficiência nos chutes a gol, geralmente têm uma maior chance de vitória.
2. **Cartões, faltas e escanteios** influenciam diretamente o controle da partida e as oportunidades de gol. Uma defesa que comete muitas faltas ou recebe cartões pode ser vulnerável, enquanto escanteios e chances ofensivas contribuem para as probabilidades de marcar gols.
3. **Desempenho médio pré-jogo (PPG e xG)**: O número de pontos por jogo e o desempenho esperado de gols (xG) são métricas que oferecem uma visão do histórico do time e indicam sua capacidade de manter o desempenho ou superá-lo em partidas futuras.

#### Abordagem Técnica:

Nesta parte foi utilizado o modelo de **Regressão Logística**, sendo aplicada para realizar as previsões de futebol. Alguns passos foram: A imputação de valores faltantes, padronização dos dados, divisão dos dados e treinamento do modelo.

- A **Regressão Logística** é um modelo estatístico usado para prever a probabilidade de um evento binário (com dois resultados possíveis) ou multiclasse.
- O modelo foi treinado com um conjunto de dados que contém estatísticas de partidas anteriores e diversas métricas de desempenho dos times.

#### Estrutura do Modelo:

- **Variáveis Preditoras**: 
  - Estatísticas da equipe da casa e da equipe visitante, incluindo posse de bola, número de chutes a gol, cartões amarelos e vermelhos, escanteios, xG (gols esperados) e faltas cometidas.
  - Dados pré-jogo como Pre-Match PPG (pontos por jogo) e Pre-Match xG (gols esperados).
- **Variável Alvo**: Resultado da partida (vitória da casa, empate ou vitória do visitante).

#### Previsão Ajustada:

O modelo prevê as probabilidades de três possíveis resultados para a partida:
- **Vitória da equipe da casa**
- **Empate**
- **Vitória da equipe visitante**

Além de fornecer as probabilidades em porcentagem, as previsões são categorizadas em níveis de probabilidade, utilizando a função `categorize_win_probability`, que classifica as chances de vitória em:
- Baixíssimas chances de ganhar
- Baixas chances de ganhar
- Médias chances de ganhar
- Muitas chances de ganhar
- Grandes chances de ganhar

Essa categorização ajuda a fornecer uma interpretação mais qualitativa das previsões feitas pelo modelo.

#### Exemplo de Funcionamento:

O modelo é capaz de prever resultados de partidas futuras, mesmo que os dados dessas partidas não estejam no conjunto de dados de treino. Nesse caso, ele utiliza **médias históricas** dos times para fazer previsões, garantindo que ainda possa fornecer uma probabilidade estimada, com base no desempenho geral dos times.


### 1.3 Criando o Modelo

#### Criando a Coluna de Resultado e Definindo Variáveis Preditoras

**Explicação**:
- `df['result']`: Estamos criando uma nova coluna chamada result que representa o resultado da partida. A classe 2 indica vitória da casa, 1 vitória do visitante, e 0 indica empate.

- `Variáveis preditoras (X) e alvo (y)`: As variáveis preditoras são características como posse de bola, chutes a gol, cartões, etc., que são usadas para treinar o modelo. A variável alvo y é o resultado da partida.



In [4]:
# Criando a coluna de resultado
df['result'] = np.where(df['home_team_goal_count'] > df['away_team_goal_count'], 2, # Vitória da casa
                        np.where(df['home_team_goal_count'] < df['away_team_goal_count'], 1, # Vitória do visitante
                                 0)) # Derrota

# Definindo a variável alvo e os preditores
y = df['result']
X = df[['home_team_encoded', 'away_team_encoded', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg',
        'home_team_goal_count', 'away_team_goal_count', 'home_team_corner_count', 'away_team_corner_count',
        'home_team_yellow_cards', 'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards',
        'home_team_shots', 'away_team_shots', 'home_team_shots_on_target', 'away_team_shots_on_target',
        'home_team_shots_off_target', 'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
        'home_team_possession', 'away_team_possession', 'Home Team Pre-Match xG', 'Away Team Pre-Match xG',
        'team_a_xg', 'team_b_xg', 'average_goals_per_match_pre_match', 'btts_percentage_pre_match',
        'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match']]


#### Imputação de Valores Faltantes, Padronização e Treinamento do Modelo

**Explicação:**

- `Imputação de valores faltantes:` Preenchemos os valores ausentes nas variáveis preditoras com a média da respectiva variável, garantindo que o modelo não falhe devido a dados incompletos.
- `Padronização dos dados:` Para evitar que as variáveis com escalas diferentes impactem negativamente o modelo, os dados são padronizados usando StandardScaler.
- `Divisão treino/teste:` O conjunto de dados é dividido em 60% para treino e 40% para teste, garantindo uma avaliação justa do modelo.
- `Logistic Regression:` Treinamos o modelo de Regressão Logística com o número máximo de 100 iterações, que são necessárias para a convergência dos solucionadores.
- `Avaliação:` As métricas de acurácia, acurácia balanceada, e a matriz de confusão nos dão uma visão do desempenho do modelo no conjunto de teste.

In [ ]:
# Imputação de valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=42)

# Criando e treinando o modelo de Regressão Logística
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

# Avaliando o modelo
y_pred = log_reg.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Acurácia Balanceada:", balanced_accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matriz de confusão: \n", confusion_matrix(y_pred, y_test))

#### Hiperparâmetros

Hiperparâmetros são parâmetros definidos antes do treinamento do modelo e que influenciam diretamente no desempenho do mesmo. Na Regressão Logística, alguns hiperparâmetros importantes são:

**Penalty (penalidade):**

- `l1:` Regularização Lasso, que tende a eliminar variáveis irrelevantes zerando seus coeficientes.
- `l2:` Regularização Ridge, que encolhe os coeficientes das variáveis sem eliminá-los completamente.

A escolha entre L1 e L2 depende da natureza dos dados. A regularização L1 é útil quando há muitas variáveis irrelevantes, enquanto a L2 é útil para evitar overfitting em modelos complexos.

**C (parâmetro de regularização):**

- O parâmetro C controla a força da regularização. Um valor menor de C implica em uma maior regularização, o que reduz a variância (overfitting) mas pode aumentar o viés. Valores altos de C, por outro lado, permitem que o modelo se ajuste mais aos dados (menor regularização), mas pode resultar em overfitting.

**Solver (solucionador):**

Define o algoritmo utilizado para ajustar o modelo. Alguns exemplos são:
- `liblinear:` Solução para problemas menores e com regularização L1 ou L2.
- `saga:` Suporta grandes datasets e regularização L1 e L2.

**GridSearchCV para ajuste de Hiperparâmetros:**

- O GridSearchCV é uma técnica que automatiza a busca pelos melhores hiperparâmetros, testando diferentes combinações dos mesmos e avaliando o desempenho para selecionar a melhor combinação. Isso ajuda a otimizar o modelo.

In [ ]:
# Definindo os parâmetros a serem testados
param_grid = {
    'penalty': ['l1', 'l2'],  # Regularização L1 ou L2
    'C': [0.01, 0.1],  # Inverso da regularização (quanto menor, mais regularização)
    'solver': ['liblinear'],  # Solvers que suportam L1 e L2
    'max_iter': [100]
}

# Criando o modelo base
log_reg = LogisticRegression(random_state=42)

# Configurando o GridSearchCV
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy')
log_reg.fit(X_train, y_train)


# Ajustando o GridSearch ao conjunto de dados
grid_search.fit(X_train, y_train)

# Exibindo os melhores parâmetros
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)

# Avaliando o modelo com os melhores parâmetros
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Avaliação do modelo otimizado
print("Acurácia após ajuste:", accuracy_score(y_test, y_pred))
print("Acurácia Balanceada após ajuste:", balanced_accuracy_score(y_test, y_pred))
print("Relatório de Classificação após ajuste:\n", classification_report(y_test, y_pred))
print("Matriz de confusão após ajuste:\n", confusion_matrix(y_pred, y_test))


## Validação Cruzada
A **validação cruzada** é uma técnica fundamental para avaliar o desempenho de modelos de aprendizado de máquina de forma mais confiável. Em vez de treinar e testar o modelo apenas uma vez com um único conjunto de divisão de dados, a validação cruzada permite que o modelo seja testado múltiplas vezes em diferentes subconjuntos do conjunto de dados, garantindo que os resultados sejam menos dependentes de uma única divisão dos dados.

Neste projeto, utilizamos a validação cruzada com **5 folds estratificados**:
- O conjunto de dados é dividido em 5 partes (folds) de forma que a proporção das classes seja mantida em cada fold (estratificação).
- Em cada iteração, o modelo é treinado em 4 folds e testado no fold restante.
- O processo é repetido 5 vezes, trocando os folds de treinamento e teste, e ao final, é calculada a média das métricas de desempenho.


In [ ]:
# Configurando a validação cruzada com 5 folds estratificados
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Realizando a validação cruzada e obtendo as previsões de probabilidade
cv_scores = cross_val_score(log_reg, X, y, cv=cv, scoring='accuracy')
cv_predictions = cross_val_predict(log_reg, X, y, cv=cv, method='predict_proba')

# Média da acurácia da validação cruzada
print(f"Acurácia média da validação cruzada: {cv_scores.mean():.4f}")

## Curva ROC e AUC para o Modelo de Regressão Logística
A **Curva ROC (Receiver Operating Characteristic)** é uma ferramenta gráfica que ilustra a capacidade de um modelo de classificação binária em distinguir entre classes positivas e negativas. A curva traça a relação entre a **Taxa de Verdadeiros Positivos (TPR)** e a **Taxa de Falsos Positivos (FPR)** para diferentes limiares de decisão.

O **AUC (Area Under the Curve)** mede a área sob a Curva ROC, fornecendo uma única métrica que reflete a capacidade do modelo de classificar corretamente as amostras. Um AUC próximo de 1 indica um modelo excelente, enquanto um AUC de 0.5 sugere que o modelo não é melhor que uma escolha aleatória.



In [ ]:
# Calculando a ROC e AUC para o modelo
# Para problemas de multiclassificação, é necessário binarizar o resultado para a curva ROC
# Vamos considerar a classe positiva como a vitória da casa, por exemplo (index 2)
y_true = y  # Labels verdadeiras
y_probs = cv_predictions[:, 2]  # Probabilidades da classe positiva (ex: vitória da casa)

# Calculando a curva ROC e o valor de AUC
fpr, tpr, _ = roc_curve(y_true, y_probs, pos_label=2)  # pos_label depende da classe positiva que está sendo analisada
roc_auc = auc(fpr, tpr)

# Plotando a curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Linha da chance (45 graus)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

# Exibindo o valor AUC da curva ROC
print(f"AUC da Curva ROC: {roc_auc:.4f}")

### Resultados

Após gerar a Curva ROC e calcular o AUC, os seguintes resultados foram obtidos:

- **Curva ROC**: A curva mostra a taxa de verdadeiros positivos (TPR) em função da taxa de falsos positivos (FPR) para diferentes limiares de decisão. O formato da curva fornece uma visão sobre o desempenho do modelo de Regressão Logística em discriminar entre classes positivas e negativas.

- **AUC (Area Under the Curve)**: O valor calculado foi de **0.94**, indicando que o modelo possui uma excelente capacidade de classificação. Um AUC próximo de 1 sugere que o modelo é muito eficaz em distinguir corretamente entre as classes.

### Conclusão

A avaliação do modelo usando a Curva ROC e AUC mostrou que o modelo tem um desempenho na tarefa de classificação:

- **Desempenho de Classificação**: A curva ROC, que se aproxima do canto superior esquerdo do gráfico, demonstra que o modelo possui uma alta taxa de verdadeiros positivos em relação aos falsos positivos, o que é ideal para um classificador binário.

- **Qualidade do Modelo**: Com um AUC de **0.94**, o modelo demonstra excelente precisão e habilidade em classificar corretamente as instâncias. Este resultado sugere que o modelo de Regressão Logística é adequado para ser utilizado no problema em questão, dado o seu alto poder discriminativo.

#### Função para Categorizar a Probabilidade de Vitória
**Explicação:**
- Função de categorização: Esta função transforma as probabilidades numéricas de vitória em descrições qualitativas, ajudando a tornar os resultados mais interpretáveis para os usuários finais.

In [9]:
# Função para categorizar a probabilidade de vitória
def categorize_win_probability(prob):
    if prob < 0.2:
        return "Baixíssimas chances de ganhar"
    elif 0.2 <= prob < 0.4:
        return "Baixas chances de ganhar"
    elif 0.4 <= prob < 0.6:
        return "Médias chances de ganhar"
    elif 0.6 <= prob < 0.8:
        return "Muitas chances de ganhar"
    elif prob >= 0.8:
        return "Grandes chances de ganhar"
    else:
        return "Erro ao categorizar"

#### Função de Previsão com Médias Históricas e Categorias de Probabilidade

**Explicação:**
- `Previsão com médias históricas:` Quando uma partida não é encontrada no conjunto de dados, o modelo utiliza as médias das estatísticas dos times para preencher as variáveis. Isso permite que o modelo faça previsões para jogos futuros ou não registrados.
- `Imputação e padronização:` Mesmo para partidas simuladas com médias, as etapas de imputação e padronização são aplicadas para garantir a consistência com o treinamento.
- `Classificação das probabilidades:` A função `categorize_win_probability` é usada para traduzir as probabilidades em categorias qualitativas (por exemplo, "Muitas chances de ganhar").

In [10]:
# Criar mapeamento dos times e IDs
home_team_map = dict(zip(df['home_team_encoded'], df['home_team_name']))
away_team_map = dict(zip(df['away_team_encoded'], df['away_team_name']))

# Função para obter o nome do time a partir do ID
def get_team_name(team_id, home=True):
    if home:
        return home_team_map.get(team_id, "Time não encontrado")
    else:
        return away_team_map.get(team_id, "Time não encontrado")

# Função para converter nome do time para o ID correspondente
def get_team_id(team_input, home=True):
    if isinstance(team_input, str):  # Se o input for uma string (nome do time)
        if home:
            return df[df['home_team_name'] == team_input]['home_team_encoded'].iloc[0]
        else:
            return df[df['away_team_name'] == team_input]['away_team_encoded'].iloc[0]
    else:
        return team_input  # Se já for um ID, retorna diretamente

# Função para prever o resultado de uma partida, independentemente de estar no CSV
def predict_match_result_with_averages(home_team_input, away_team_input):
    global imputer, scaler, log_reg
    
    # Converter input para IDs se necessário
    home_team_encoded = get_team_id(home_team_input, home=True)
    away_team_encoded = get_team_id(away_team_input, home=False)

    # Obter os nomes dos times a partir dos IDs
    home_team_name = get_team_name(home_team_encoded, home=True)
    away_team_name = get_team_name(away_team_encoded, home=False)

    print(f"Previsão para o jogo: {home_team_name} vs {away_team_name}")
    
    # Verificar se a partida está no dataset
    match_data = df[(df['home_team_encoded'] == home_team_encoded) & (df['away_team_encoded'] == away_team_encoded)].copy()
    
    if match_data.empty:
        print("Jogo não encontrado no conjunto de dados. Usando médias históricas dos times.")
        
        # Calcular médias das estatísticas dos times para criar uma nova entrada
        home_team_stats = df[df['home_team_encoded'] == home_team_encoded].select_dtypes(include=[np.number]).mean()
        away_team_stats = df[df['away_team_encoded'] == away_team_encoded].select_dtypes(include=[np.number]).mean()

        # Criar uma nova entrada com as estatísticas médias, preenchendo as colunas de gols com zero
        match_data = pd.DataFrame({
            'Pre-Match PPG (Home)': [home_team_stats['Pre-Match PPG (Home)']],
            'Pre-Match PPG (Away)': [away_team_stats['Pre-Match PPG (Away)']],
            'home_ppg': [home_team_stats['home_ppg']],
            'away_ppg': [away_team_stats['away_ppg']],
            'home_team_corner_count': [home_team_stats['home_team_corner_count']],
            'away_team_corner_count': [away_team_stats['away_team_corner_count']],
            'home_team_yellow_cards': [home_team_stats['home_team_yellow_cards']],
            'home_team_red_cards': [home_team_stats['home_team_red_cards']],
            'away_team_yellow_cards': [away_team_stats['away_team_yellow_cards']],
            'away_team_red_cards': [away_team_stats['away_team_red_cards']],
            'home_team_shots': [home_team_stats['home_team_shots']],
            'away_team_shots': [away_team_stats['away_team_shots']],
            'home_team_shots_on_target': [home_team_stats['home_team_shots_on_target']],
            'away_team_shots_on_target': [away_team_stats['away_team_shots_on_target']],
            'home_team_shots_off_target': [home_team_stats['home_team_shots_off_target']],
            'away_team_shots_off_target': [away_team_stats['away_team_shots_off_target']],
            'home_team_fouls': [home_team_stats['home_team_fouls']],
            'away_team_fouls': [away_team_stats['away_team_fouls']],
            'home_team_possession': [home_team_stats['home_team_possession']],
            'away_team_possession': [away_team_stats['away_team_possession']],
            'Home Team Pre-Match xG': [home_team_stats['Home Team Pre-Match xG']],
            'Away Team Pre-Match xG': [away_team_stats['Away Team Pre-Match xG']],
            'team_a_xg': [home_team_stats['team_a_xg']],
            'team_b_xg': [away_team_stats['team_b_xg']],
            'average_goals_per_match_pre_match': [(home_team_stats['average_goals_per_match_pre_match'] + away_team_stats['average_goals_per_match_pre_match']) / 2],
            'btts_percentage_pre_match': [(home_team_stats['btts_percentage_pre_match'] + away_team_stats['btts_percentage_pre_match']) / 2],
            'average_corners_per_match_pre_match': [(home_team_stats['average_corners_per_match_pre_match'] + away_team_stats['average_corners_per_match_pre_match']) / 2],
            'average_cards_per_match_pre_match': [(home_team_stats['average_cards_per_match_pre_match'] + away_team_stats['average_cards_per_match_pre_match']) / 2],
            'home_team_goal_count': [0],  # Preencher com 0 porque é uma partida futura
            'away_team_goal_count': [0],  # Preencher com 0 porque é uma partida futura
            'home_team_encoded': [home_team_encoded],
            'away_team_encoded': [away_team_encoded]
        })

    # Certificar-se de que as colunas do novo conjunto de dados correspondem às do conjunto de treino
    match_data = match_data[X.columns]
    
    # Imputação e padronização dos dados
    match_data_imputed = imputer.transform(match_data)
    match_data_scaled = scaler.transform(match_data_imputed)
    
    # Prevendo as probabilidades para cada resultado (vitória da casa, empate, vitória do visitante)
    probabilities = log_reg.predict_proba(match_data_scaled)[0]
    
    # Extraindo as probabilidades para cada resultado
    home_win_prob = probabilities[2]
    draw_prob = probabilities[0]
    away_win_prob = probabilities[1]
    
    # Classificar as probabilidades usando a função categorize_win_probability
    home_team_chance = categorize_win_probability(home_win_prob)
    away_team_chance = categorize_win_probability(away_win_prob)
    
    # Exibir o resultado formatado com as categorias
    result = {
        'Probabilidade de vitória da casa (%)': home_win_prob * 100,
        'Chance de vitória da casa': home_team_chance,
        'Probabilidade de empate (%)': draw_prob * 100,
        'Probabilidade de vitória do visitante (%)': away_win_prob * 100,
        'Chance de vitória do visitante': away_team_chance
    }
    
    # Exibir o resultado formatado
    for key, value in result.items():
        print(f"{key}: {value}")
    
    return result

#### Exemplo de Uso
**Explicação:**
- `Exemplo prático:` Aqui, aplicamos a função de previsão para um jogo hipotético entre dois times. As probabilidades de vitória para ambos os times e a classificação qualitativa são exibidas.

In [ ]:
# Exemplo de uso
# home_team_id = 13
# away_team_id = 10
resultado = predict_match_result_with_averages(home_team_input='Palmeiras', away_team_input='Atlético Mineiro')

#### 1.4 Métricas do modelo
***Explicação:***

Abordamos o uso das métricas, isto é, a avaliação de desempenho das predições. Usamos o submódulo metrics do Scikit-Learn, para conseguir acessar as funções de acurácia, acurácia balanceada, precisão, recall, F1-Score e a Matriz de Confusão. Nesse sentido, ao observar o conjunto de dados, é possível concluir que a grande maioria dos dados são compostos por empates, oque configura na condição de "classes desbalanceadas". Em vista disso, focamos em aumentar as porcentagens de acurácia balanceada e F1-Score, que são duas métricas que são ideais ao enfrentar esse tipo de dados.